In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px 
print(plotly.__version__)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Heart Failure Prediction Dataset

## Problem
심장 혈관의 질환들(Cardiovascular diseases)(CVDs)은 죽음의 원인 중 1번째로 전세계적으로 많이 발생한다. 이는 매년 17.9만 명에 이를 정도로 추정되며, 이는 전세계 죽음의 31%에 해당한다.

CVD 사망자 5명 중 4명은 심장마비와 뇌졸중으로 사망하며, 이 중 3분의 1은 70세 이하에서 조기 사망한다. 심부전은 CVD로 인해 발생하는 일반적인 일이며, 이 데이터셋에는 가능한 심장질환을 예측하는 데 사용할 수 있는 11가지 기능이 포함되어 있습니다.

심혈관 질환이 있거나 (고혈압, 당뇨병 등 이미 질병이 하나 이상의 위험 인자의 존재로 인해) 심혈관 위험이 높은 사람은 ML모델이 큰 도움이 될 수 있는 조기 발견과 관리가 필요하다.

## Attribute description

https://en.wikipedia.org/wiki/ST_depression

1. Age : 환자의 나이 (years)
2. Sex : 환자의 성별 [M=Male, F=Female]
3. ChestPainType : 가슴 통증 유형 [TA=특정 협심증, ATA=무정형한 협심증, NAP=협심증 통증 없음, ASY=증상없음]
4. RestingBP : 휴식기 혈압 (mm Hg)
5. Cholesterol(콜레스테롤) : 혈청 콜레스테롤 (mm/dl)
6. FastingBS : 단식 혈당 [1: 120이상 mg/dl, 0: 그렇지 않음]
7. RestingECG : 휴식기 심전도 결과 
    - Normal: 정상
    - ST: having ST-T 파동 비정상성 (T 파동 역치 and/or ST 상승 또는 하강 of > 0.05 mV)
    - LVH: showing probable or definite 좌심실 비대증
8. MaxHR: 도달한 최대 심박수 최대값 [60 ~ 202 사이의 수치 값]
9. ExerciseAngina: 활동 원인이 되는 협심증 [Y: Yes, N: No]
10. Oldpeak: ST [ST 하강에서 측정된 수치 값]
11. ST_Slope: 영향을 미치는 ST segment 정상의 기울기 [Up: 오르막, Flat: 평평, Down: 내리막]
12. HeartDisease: output class [1: 심장병, 0: 정상]


In [2]:
df = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')

In [3]:
df.info()

# EDA

In [4]:
df.sample(5)

- Sex, ChestPainType, RestingECG, ExerciseAngina, ST_Slope (object type) = one-hot encoding

In [5]:
df.describe()

- Age : 28 ~ 77 세. (평균 53 세)(-> hist 시각화)
- RestingBP	Cholesterol	FastingBS : min으로 0인 값이 확인 (-> 결측치 인가?)
- HeartDisease : 심장별 클래스는 O:X=55:45 비율로 존재

In [6]:
# https://wikidocs.net/92114

plt.figure(figsize=(20,4))
i = 1
cat_name = []
for name in df.columns:
    if df[name].dtype == 'object':
        cat_name.append(name)
        ax = plt.subplot(1, 5, i)
        ax.set_title(name)
        
        data = df[name].value_counts()
        plt.pie(x=data.values/len(df), labels=data.index, startangle=260, autopct='%.1f%%')
        i += 1
plt.show()
cat_name

- 남성(Male):여성(Female) = 79%:21%
- 가슴통증 유형에서 증상없음(ASY)이 54%, 협심증이 아닌 통증(NAP)이 22%, 비정형적인 협심증(ATA)이 18%, 전형적인 협심증(TA)이 5%를 차지함.
- 심전도 결과에서는 정상(Normal) 60%, LVH 와 ST 가 각각 20%로 비슷한 빈도로 나타난다.
- 활동하는 협심증은 없음(No) 상태가 59%, 있음(Yes)가 40% 로, 없는 상태가 살짝 더 많았음.
- ST 기울기에서는 평평(Flat)한 경우가 50%, 오르막(Up)인 경우가 43%, 내리막(Down)인 경우가 6%로, 절반은 평평하거나 오르막 상태인 경우가 많았다.

# Str Encoding

In [7]:
# Or LabelEncoder() 같은 작업함

mapping = {'Sex': {'M':0, 'F':1},
           'ChestPainType': {'ASY':0, 'NAP':1, 'ATA':2, 'TA':3},
           'RestingECG': {'Normal':0, 'LVH':1, 'ST':2},
           'ExerciseAngina': {'Y':0, 'N':1},
           'ST_Slope': {'Up':0, 'Flat':1, 'Down':2}
          }

for name in df.columns:
    if name in mapping:
        df[name] = df[name].map(mapping[name])

# EDA

In [8]:
# df.corr()

fig, ax = plt.subplots(1,1 ,figsize=(10, 6))
sns.heatmap(df.corr(), ax=ax,
            vmin=0, vmax=1, center=0,
            cmap='coolwarm',
            annot=True, fmt='.2f')
plt.show()

In [9]:
# !pip install plotly statsmodels

In [10]:
# fig = px.parallel_coordinates(df, color='HeartDisease')
# fig.show()

- MaxHR 값이 높을 수록, Age가 어릴 수록 HeartDisease 가 아닐 확률이 더 높음.

(-> 나머지 str로 numerial encoding 해서, 다시 시각화)

In [11]:
fig, ax = plt.subplots(figsize=(12, 7))

sns.histplot(x='Age', data=df, ax=ax,
#              binwidth=50, 
#              bins=100,
            )

plt.show() # 나이 50 ~ 60 대가 가장 분포

In [12]:
fig, ax = plt.subplots(figsize=(10, 7))

data = df['FastingBS'].value_counts()
plt.bar(data.index, data.values, align='center', tick_label=[0,1])
plt.show()

In [13]:
fig, ax = plt.subplots(figsize=(12, 7))

sns.histplot(x='RestingBP', data=df, ax=ax,
#              binwidth=50, 
#              bins=100,
            )

plt.show() # 대게 90 ~ 200 사이 값이므로, 0은 결측치 같음. 어떻게 처리할까? 변수 특성?

In [14]:
fig, ax = plt.subplots(figsize=(12, 7))

sns.histplot(x='Cholesterol', data=df, ax=ax,
#              binwidth=50, 
#              bins=50,
            )

plt.show() # 대게 100 ~ 400 사이의 값이고, 0이 가장 많음. 잘 모르는 경우인 것 같은데, 현재 분포대로 처리할까? 

In [15]:
fig, ax = plt.subplots(figsize=(12, 7))

sns.histplot(x='MaxHR', data=df, ax=ax,
#              binwidth=50, 
#              bins=100,
            )

plt.show()

In [16]:
fig, ax = plt.subplots(figsize=(12, 7))

sns.histplot(x='Oldpeak', data=df, ax=ax,
#              binwidth=50, 
#              bins=100,
            )

plt.show()

# Missing Value

In [17]:
# data = df['RestingBP'][df['RestingBP'] != 0]

# df['RestingBP'] = df['RestingBP'].apply(lambda x: x if x != 0 else np.mean(data))

In [18]:
# data = df['Cholesterol'][df['Cholesterol'] != 0]
# n = len(df) - len(data)

# s = iter(np.random.gamma(np.sqrt(np.mean(data)), 2*np.sqrt(np.std(data)), n))
# df['Cholesterol'] = df['Cholesterol'].apply(lambda x: x if x != 0 else next(s))

# Train & Test dataset split

In [19]:
X, y = df.drop(labels=['HeartDisease'], axis=1), df['HeartDisease']

In [20]:
from sklearn.model_selection import train_test_split

SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=SEED, 
                                                    shuffle=True, stratify=y)

# Model 1.  Catboost + grid_search

- 결측치에도 잘 예측 
- H.P 튜닝 안해도 영향이 크지X
- Table data에 적합(?)

In [21]:
# https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier

from catboost import Pool, CatBoostClassifier

EPOCH = 100
LR = 1e-1
MAX_DEPTH = 4
loss_fn = 'CrossEntropy' # 'Logloss'

# train_ds = Pool(X_train, y_train)
# test_ds = Pool(X_test, y_test)

model = CatBoostClassifier(loss_function=loss_fn, verbose=False)

grid = {'iterations' : [100, 150],
        'learning_rate': [1, 5e-1, 1e-1],
        'depth': [2, 4, 6, 8, 10]}

grid_search_result = model.grid_search(grid,
                                       X=X_train, 
                                       y=y_train,
                                       plot=True)

# model.fit(train_ds)

In [22]:
from sklearn.metrics import accuracy_score

preds = model.predict(X_test)
accuracy_score(preds, y_test)

# Model 2.  Decision Tree

- 상관관계가 있는 열들이 여러 개

In [23]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=3, random_state=42)
clf = tree.fit(X_train, y_train)

In [24]:
preds = tree.predict(X_test)
accuracy_score(preds, y_test)

# Model 3. Random Forest (Ensemble)

In [25]:
n_estimators = 30
max_depth =6
seed = 42

In [26]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# max_depth=6
random_forest = RandomForestClassifier(
    n_estimators=n_estimators, max_depth=max_depth, random_state=seed
)
random_forest = random_forest.fit(X_train, y_train)

# gradient_boosting = GradientBoostingClassifier(
#     n_estimators=n_estimators, max_depth=5, random_state=seed
# )
# gradient_boosting = gradient_boosting.fit(X_train, y_train)

In [27]:
preds = random_forest.predict(X_test)
print(accuracy_score(preds, y_test))

# preds = gradient_boosting.predict(X_test)
# print(accuracy_score(preds, y_test))

# Model 4. SVC

In [28]:
from sklearn.svm import SVC

svc = SVC(C=3000, kernel='rbf')
svc.fit(X_train, y_train)
[]
preds = svc.predict(X_test)
accuracy_score(preds, y_test)

In [29]:
from sklearn.linear_model import LogisticRegressionCV

logi_reg = LogisticRegressionCV(max_iter=50000, cv=5, random_state=42)
logi_reg.fit(X_train, y_train)

preds = logi_reg.predict(X_test)
accuracy_score(preds, y_test)

# XAI

## Catboost

In [30]:
import shap
shap.initjs()

In [31]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# visualize the first prediction's explanation
shap.force_plot(explainer.expected_value, shap_values[40,:], X.iloc[40,:])

In [32]:
# visualize the training set predictions
shap.force_plot(explainer.expected_value, shap_values, X)

In [33]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X)

## Random forest

In [34]:
from sklearn import tree

plt.figure(figsize=(20, 10))
_ = tree.plot_tree(random_forest.estimators_[0], feature_names=X.columns, filled=True, fontsize=13)

In [36]:
# from lime import lime_tabular

# explainer = lime_tabular.LimeTabularExplainer(X, mode="classification", feature_names= X.columns)
# explanation = explainer.explain_instance(X_train[0], model.predict_proba, num_features=len(X.columns), labels=(0,1) ,num_samples=5000)

# explanation.show_in_notebook()